# JSONPath 解析 JSON 数据

本笔记系统整理在 Python 中使用 **JSONPath** 提取嵌套 JSON 数据的方式，涵盖安装、基础语法、进阶 (通配符/递归/过滤/切片)、错误与性能处理、与手动遍历对比，以及一个教学用的迷你实现。所用主库：`jsonpath-ng`。
```shell
pip install jsonpath-ng
```



## Python JSONPath 库对比：jsonpath vs jsonpath-rw vs jsonpath-ng

| 特性            | **jsonpath**     | **jsonpath-rw**           | **jsonpath-ng**           |
| :------------ | :--------------- | :------------------------ | :------------------------ |
| **维护状态**      | ❌ **已停止维护**      | ⚠️ 基本维护                   | ✅ **活跃维护**                |
| **实现方式**      | 简单函数式            | 完整语言实现 (AST)              | 完整语言实现 (AST)              |
| **依赖项**       | **零依赖**（单文件）     | 多个依赖（ply, six等）           | 继承自 jsonpath-rw           |
| **语法支持**      | 基础 JSONPath + 扩展 | 完整语法 + 扩展操作符              | **最全面**（合并 rw-ext）        |
| **特殊功能**      | 点号索引扩展 `$.key.0` | 命名操作符（`parent`）           | **支持更新/删除节点**             |
| **Python 支持** | 2.x/3.x（旧）       | 2.7, 3.4+                 | **3.8+**（现代）              |
| **安装方式**      | 需手动下载            | `pip install jsonpath-rw` | `pip install jsonpath-ng` |

## 1. 示例数据
构造一个较为真实的嵌套结构，包括：
- 用户数组 (id, name, roles, profile, orders)
- orders 内含 items 数组
- meta 区域含 flags 布尔标记
便于覆盖各种查询。

In [35]:
data = {
    'users': [
        {
            'id': 1,
            'name': 'Alice',
            'roles': ['admin', 'editor'],
            'profile': {'age': 30, 'email': 'alice@example.com'},
            'orders': [
                {'order_id': 'O100', 'amount': 120.5, 'items': [ {'sku': 'A1', 'qty': 2}, {'sku': 'B7', 'qty': 1} ]},
                {'order_id': 'O101', 'amount': 15.0, 'items': []}
            ]
        },
        {
            'id': 2,
            'name': 'Bob',
            'roles': ['viewer'],
            'profile': {'age': None, 'email': 'bob@example.com'},
            'orders': [
                {'order_id': 'O102', 'amount': 300.0, 'items': [ {'sku': 'X9', 'qty': 5} ]}
            ]
        }
    ],
    'meta': {
        'generated_at': '2025-11-13T12:00:00Z',
        'source': 'test_system',
        'flags': [ {'name': 'beta', 'enabled': True}, {'name': 'deprecated', 'enabled': False} ]
    }
}
len(data['users']), len(data['meta']['flags'])

(2, 2)

## 2. 导入与基本使用
使用 `parse(expr).find(data)` 返回匹配对象列表，每个对象具备 `.value` 属性。

In [36]:
from jsonpath_ng import parse
expr = parse('$.users[*].name')
matches = expr.find(data)
[m.value for m in matches]

['Alice', 'Bob']

## 3. 辅助函数简化输出
封装一个小工具便于快速实验表达式。

In [37]:
def run(expr, data=data):
    jp = parse(expr)
    matches = jp.find(data)
    values = [m.value for m in matches]
    print(expr, '->', values if values else '[] (空)')
    return values
run('$.users[0].profile.email')
run('$.users[*].orders[*].order_id')
run('$.meta.flags[*].name')

$.users[0].profile.email -> ['alice@example.com']
$.users[*].orders[*].order_id -> ['O100', 'O101', 'O102']
$.meta.flags[*].name -> ['beta', 'deprecated']


['beta', 'deprecated']

## 4. 通配符 / 递归下降
- `*`：匹配任意 key 或数组所有元素。
- `..`：递归搜索所有层级 (谨慎使用，可能性能较低)。

In [38]:
run('$.users[*].profile.*')  # profile 下所有值
run('$..order_id')           # 所有层级 order_id
run('$..qty')                # 所有商品数量
run('$..flags[*].enabled')   # 所有 flag 的 enabled

$.users[*].profile.* -> [30, 'alice@example.com', None, 'bob@example.com']
$..order_id -> ['O100', 'O101', 'O102']
$..qty -> [2, 1, 5]
$..flags[*].enabled -> [True, False]


[True, False]

## 5. 过滤 (Filter)
语法：`[?(条件)]`，内部使用 `@` 指代当前元素。支持比较、`in`、逻辑运算等。

In [39]:
# ❌ 错误导入（基础解析器）
# from jsonpath_ng import parse

# ✅ 正确导入（扩展解析器，支持过滤器、正则等）
from jsonpath_ng.ext import parse

def run(expr, data=data):
    jp = parse(expr)  # 现在可以正确解析 ? 过滤器
    matches = jp.find(data)
    values = [m.value for m in matches]
    print(f"{expr:<55} -> {values}")
    return values

run('$.users[?(@.profile.age != null)].name')            # 年龄非空用户
run('$.users[*].orders[?(@.amount > 100)].order_id')     # 金额 > 100 的订单
# run('$.users[?("admin" in @.roles)].name')             # 具有 admin 角色的用户名  ??? 目前报错
run('$.users[*].orders[?(@.items)].order_id')           # items 非空的订单 (若支持)


$.users[?(@.profile.age != null)].name                  -> ['Alice', 'Bob']
$.users[*].orders[?(@.amount > 100)].order_id           -> ['O100', 'O102']


JsonPathParserError: Parse error at 1:18 near token in (ID)

### 两种解析器对比
| 特性               | `jsonpath_ng.parse` | `jsonpath_ng.ext.parse`  |
| :--------------- | :------------------ | :----------------------- |
| **基础路径**         | ✅ 支持                | ✅ 支持                     |
| **过滤器 `?(...)`** | ❌ 不支持               | ✅ **完全支持**               |
| **`in` 运算符**     | ❌ 不支持               | ✅ 支持                     |
| **正则匹配 `=~`**    | ❌ 不支持               | ✅ 支持                     |
| **函数调用**         | ❌ 不支持               | ✅ 支持（`sum()`, `len()` 等） |
| **使用场景**         | 简单查询                | 复杂查询、生产环境                |


## 6. 数组切片
切片格式通常与 Python 类似：`[start:end:step]`。有些实现可能仅支持部分。

In [28]:
run('$.users[0:2].name')    # 前两个用户
run('$.users[0].roles[0:2]') # 第一个用户的前两个角色

$.users[0:2].name                                       -> ['Alice', 'Bob']
$.users[0].roles[0:2]                                   -> ['admin', 'editor']


['admin', 'editor']

## 7. 复杂查询示例
组合多层嵌套：

In [29]:
run('$.users[*].orders[*].items[?(@.qty > 2)].sku')   # 数量 >2 的 SKU
run('$.meta.flags[?(@.enabled)].name')                # 启用的 flags 名称
run('$.users[?(@.orders)].name')                      # 有订单的用户名 (orders 非空)

$.users[*].orders[*].items[?(@.qty > 2)].sku            -> ['X9']
$.meta.flags[?(@.enabled)].name                         -> ['beta', 'deprecated']
$.users[?(@.orders)].name                               -> ['Alice', 'Bob']


['Alice', 'Bob']

## 8. 错误与边界处理
- 路径不匹配 => 空列表
- 解析表达式错误 => 抛异常
- 缺失字段与 None 区别：缺失 => 不返回；字段存在值为 None => 返回 None。

In [30]:
try:
    run('$.users[*].unknown')
    parse('??')
except Exception as e:
    print('表达式错误捕获:', e)
empty = run('$.users[?(@.profile.age > 200)].id')  # 不满足条件
print('空结果长度:', len(empty))

$.users[*].unknown                                      -> []
表达式错误捕获: Parse error at 1:0 near token ? (?)


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

## 9. 性能与预编译
重复使用同一表达式时将其 parse 一次缓存，可减少解析开销。递归 `..` 与复杂过滤在大 JSON 上要谨慎。

In [ ]:
import time
expr_all_orders = parse('$..order_id')
start = time.perf_counter()
for _ in range(1000):
    _ = [m.value for m in expr_all_orders.find(data)]
dur = time.perf_counter() - start
print('预编译重复1000次耗时: %.4f s' % dur)
start = time.perf_counter()
for _ in range(1000):
    expr_tmp = parse('$..order_id')
    _ = [m.value for m in expr_tmp.find(data)]
dur2 = time.perf_counter() - start
print('每次重新 parse 耗时: %.4f s' % dur2)

## 10. 手动遍历对比
当逻辑复杂或需要统计/聚合时，手写遍历可能更直观也更快。

In [ ]:
# 手动收集所有 order_id
manual = []
for u in data['users']:
    for o in u['orders']:
        manual.append(o['order_id'])
jsonpath_result = run('$..order_id')
print('一致性:', sorted(manual) == sorted(jsonpath_result))

## 11. 实用封装函数
为常用查询提供默认值/首个值模式。

In [ ]:
def query_json(data, expr, default=None, first=False, strict=False):
    """执行 JSONPath 查询。
    参数: data: 对象; expr: JSONPath 字符串; default: 空或异常时默认; first: 仅返回首值; strict: 表达式错误时是否抛出。
    返回: 列表或首值或 default。
    """
    try:
        jp = parse(expr)
        matches = jp.find(data)
        values = [m.value for m in matches]
        if first:
            return values[0] if values else default
        return values if values else (default if default is not None else [])
    except Exception as e:
        if strict:
            raise
        return default
print(query_json(data, '$.users[*].name'))
print(query_json(data, '$.users[?(@.profile.age != null)].name', first=True))
print(query_json(data, '$.bad.path', default='N/A'))
print(query_json(data, '??', default='Err', strict=False))

## 12. 教学用迷你 JSONPath (子集)
仅支持：`$.key1.key2`、数组索引 `[n]`、通配符 `*` 单层。不含过滤/递归。

In [ ]:
def mini_jsonpath(data, path):
    if not path.startswith('$.'):
        raise ValueError('路径需以 $. 开头')
    tokens = []
    buf = path[2:]
    # 简易分割：按 . 再处理 [index]
    parts = buf.split('.') if buf else []
    for p in parts:
        while '[' in p:
            left, rest = p.split('[', 1)
            if left:
                tokens.append(left)
            idx, after = rest.split(']', 1)
            tokens.append(int(idx))
            p = after
        if p:
            tokens.append(p)
    # 评估
    results = [data]
    for t in tokens:
        next_res = []
        for r in results:
            if t == '*':
                if isinstance(r, dict):
                    next_res.extend(r.values())
                elif isinstance(r, list):
                    next_res.extend(r)
            elif isinstance(t, int):
                if isinstance(r, list) and 0 <= t < len(r):
                    next_res.append(r[t])
            else:  # 字符串 key
                if isinstance(r, dict) and t in r:
                    next_res.append(r[t])
        results = next_res
    return results
mini_jsonpath(data, '$.users[0].profile.email'), mini_jsonpath(data, '$.users[*].id')

## 13. 迷你实现测试
与标准库结果对比。

In [ ]:
print('标准:', run('$.users[0].profile.email'))
print('迷你 :', mini_jsonpath(data, '$.users[0].profile.email'))
print('标准:', run('$.users[*].id'))
print('迷你 :', mini_jsonpath(data, '$.users[*].id'))

## 14. 常见陷阱与最佳实践
1. 不要过度依赖递归 `..`，会扫描全部层级。
2. 复杂条件可先用 JSONPath 粗选，再用 Python 过滤。
3. 预编译表达式复用降低解析开销。
4. 对空结果做显式处理 (返回默认值/抛异常)。
5. 记录查询表达式方便调试与可追溯性。

## 15. 进一步资源
- IETF JSONPath 规范草案
- jsonpath-ng 项目文档
- 在线 JSONPath 测试工具 (多个网站)
- 与 JMESPath、jq 的对比：JMESPath 更偏向结构化查询与转换；jq 在命令行强大。